## 載入資料

In [1]:
import numpy as np
import codecs
import jieba
import jieba.posseg as pseg
import re
import operator
import json
jieba.set_dictionary('./Workspace/sentiment/0616/big/jieba356726.txt')
jieba.load_userdict('./Workspace/sentiment/0616/big/jieba356726.txt')
jieba.load_userdict('./Workspace/sentiment/0616/big/cute.txt')
jieba.load_userdict('./Workspace/sentiment/0616/food/fooddict2027.txt')
jieba.load_userdict('./Workspace/sentiment/0616/menu/menu50806_new.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/negativewords.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/positivewords.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/negative.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/more.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/question.txt')
jieba.load_userdict('./Workspace/sentiment/0616/stop/stopword2292.txt')
# 負面
negdict = []
# 正面
posdict = []
# 否定
nodict = []
# 程度
plusdict = []
# 不肯定
question = []
with codecs.open('./expand/negativeWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        negdict.append(w.split()[0])
with codecs.open('./expand/positiveWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        posdict.append(w.split()[0])
with codecs.open('./expand/inverseWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        nodict.append(w.split()[0])
with codecs.open('./expand/degreeWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        plusdict.append(w.split()[0])
with codecs.open('./Workspace/sentiment/0616/sentiment/question.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        question.append(w.split()[0])

Building prefix dict from /Users/fan/anaconda/bin/Workspace/sentiment/0616/big/jieba356726.txt ...
DEBUG:jieba:Building prefix dict from /Users/fan/anaconda/bin/Workspace/sentiment/0616/big/jieba356726.txt ...
Dumping model to file cache /var/folders/ws/qq63c39d43l_20sybqcwcf900000gn/T/jieba.u60b07a305259704c4cd827282b04b44e.cache
DEBUG:jieba:Dumping model to file cache /var/folders/ws/qq63c39d43l_20sybqcwcf900000gn/T/jieba.u60b07a305259704c4cd827282b04b44e.cache
Loading model cost 1.865 seconds.
DEBUG:jieba:Loading model cost 1.865 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [2]:
import codecs
food = set()
with codecs.open('./Workspace/sentiment/0616/menu/menu50806_new.txt', 'r','utf-8') as f:
    for w in f.readlines():
        food.add(w.replace(' 5000 n\n',''))
with codecs.open('./Workspace/sentiment/0616/food/fooddict2027.txt', 'r','utf-8') as f:
    for w in f.readlines():
        food.add(w.replace(' 5000 n\n',''))
print len(list(food))

52226


In [65]:
foodlist = []
for ele in food:
    jb = list(jieba.cut_for_search(ele))
    foodlist.extend(jb)
foodlist = [ele for ele in list(set(foodlist)) if ele not in food]

In [86]:
food_from_kmean = ['7','8','15','24','32','34','36','37','42','43','53','54','65','84','86','87','97','99','103','106','110','117','118','122','123','127','140','142','145']
with open('./Workspace/data/subjectClustering_minCount_1_n_150.json', 'r') as f:
    kmean_result = json.load(f)
food_list = []
for ele in kmean_result:
    if ele in food_from_kmean:
        food_list.extend(kmean_result[ele])
print len(food_list)

8199


In [87]:
food_extend = [ele for ele in food_list if ele not in food]
print len(food_extend)

3053


In [96]:
with open('./Workspace/data/extend_word.txt', 'w') as f:
    for ele in food_extend:
        f.write(ele.encode('utf-8') + '\n')

## 建立情緒分析類別

In [3]:
class SentimentAnalysis():
    
    '''建立instance時即讀取字典'''
    def __init__(self, negdict, posdict, nodict, plusdict, question, bid, aid, food):
        self.score = 0
        self._id = bid
        self.author_id = aid
        self.positiveSubject = []
        self.negativeSubject = []
        self.negdict = negdict
        self.posdict = posdict
        self.nodict = nodict
        self.plusdict = plusdict
        self.question = question
        self.food = food
        self.adjindex = 0
        self.pList = []
        self.nList = []
    
    ''' 
        [情緒分數]
        
        計算情緒分數時，先考慮情緒詞正負，再考慮程度與極性      
        例如：好吃的義大利麵 => +1
             不好吃的義大利麵 => -1
             很好吃的義大利麵 => +2
             很不好吃的義大利麵 => -2
             
        考量大部分人傾向用詞委婉，情緒詞正負方向採用不同的處理方式
        對於負評的認定較為寬鬆，而對於正評的認定較為嚴格      
        例如：這個漢堡不算難吃 => 0
             這個漢堡不算好吃 => -1
             
        當否定詞與情緒詞中間夾雜程度用語時，皆判定為中性偏負           
        例如：這個漢堡不算非常難吃 => -0.5
             這個漢堡不算非常好吃 => -0.5
    '''
    def checkDegree(self, sdrange, positive=True):
        # 定義初始極性
        disturbingWords = [
            u'照到',
            u'照過',
            u'見過',
            u'見到',
            u'看過',
            u'看到',
            u'聽過',
            u'聽到',
            u'吃過',
            u'吃到',
            u'用過',
            u'用到',
            u'喝過',
            u'喝到',
            u'聞過',
            u'聞到',
            u'碰過',
            u'碰到',
            u'遇過',
            u'遇到',
            u'去過',
            u'貼過',
            u'拍過',
            u'拍到',
            u'推過',
            u'試過'
        ]
        if positive:
            pollar = 1
        else:
            pollar = -1
        # 定義初始程度
        degree = 1
        # 在指定範圍內撈否定詞、程度詞
        for i in xrange(len(sdrange)):
            st = sdrange[i]
            # 標記程度用語是否出現在否定詞之前
            flag = False
            # 標記程度用語是否穿插在否定詞與情緒詞之間
            ambiguous = False
            # 標記是否為負面情緒詞前面含負面詞而不含程度詞
            excp = False
            # 遇到否定用語就改變極性方向
            if st in self.nodict:
                if i == len(sdrange)-1:
                    self.adjindex = len(sdrange)-i
                    pollar *= -1
                elif sdrange[i+1] not in disturbingWords:
                    self.adjindex = len(sdrange)-i
                    pollar *= -1
            # 若含程度用語
            if st in self.plusdict:
                flag = True
                # 在程度用語前面的範圍找否定詞
                for j in xrange(i):
                    if sdrange[j] in self.nodict and sdrange[j+1] not in disturbingWords:
                        flag = False
                        # 找到代表程度用語穿插在否定詞語情緒詞之間，予以標記
                        ambiguous = True
                        break
                # 若程度用語出現在否定詞之前則乘數設為2
                if flag:
                    degree = 2
                # 若程度用語穿插在否定詞語情緒詞之間，正向詞乘數為-0.5，負向詞乘數為0.5，最後各自乘上極性後分數同為-0.5
                elif ambiguous:
                    if pollar == 1 :
                        degree = -0.5
                    else:
                        degree = 0.5
            # 負向詞遇到否定詞，若沒有程度詞修飾，直接認定為中性
            elif not positive:
                for x in xrange(len(sdrange)):
                    if sdrange[x] in self.nodict:
                        excp = True
                        break
        if excp:
            return 0
        else:
            return pollar * degree
        
    '''
        [情緒分數加總]
        
        找到情緒詞後往前找3個詞，把3個詞丟進checkDegree判斷規則，回傳該情緒詞的分數
        每次計算完分數，判斷正負號，再執行getSubject去抓取正負主題詞，結果存入self.positiveSubject/self.negativeSubject
        並把每次的計算分數累加存入self.score
        
    '''
    def sentimentScore(self, sd):
        words = sd
        self.pList = []
        self.nList = []
        # 含這些髒東西的句子之後會被跳過
        #pattern = u'文章|本文'
        # 搜尋情緒詞時用倒著找的方式，便於之後主題詞的抓取
        for i in xrange(len(words)):
            self.adjindex = 0
            p = 0
            possbj = None
            negsbj = None
            #遇到髒東西或者是出現使整個句子情緒方向不定的字眼時直接跳過 ex: 見仁見智
            #if re.search(pattern, ''.join(words)) or words[i] in self.question:
            #    break
            # 定義選取範圍
            if i > 2:
                n = 3
            else:
                n = i
            # 若出現在負面詞
            if words[i] in self.negdict:
                if n > 0:
                    p = self.checkDegree(words[i-n:i],positive=False)
                    self.score += p
                else:
                    p = -1
                    self.score += p
            # 若出現在正面詞
            elif words[i] in self.posdict:
                if n > 0:
                    p = self.checkDegree(words[i-n:i],positive=True)
                    self.score += p
                else:
                    p = 1
                    self.score += p
            # 分別抓取正負情緒詞所對應的主題詞，存入positiveSubject/negativeSubject
            # 主題詞只保留2個字以上的
            if p > 0:
                possbj = self.getSubject(i-self.adjindex, sd)
                if possbj and len(possbj)>1:
                    self.pList.append((possbj, words[i]))
            elif p < 0:
                negsbj = self.getSubject(i-self.adjindex, sd)
                if negsbj and len(negsbj)>1:
                    self.nList.append((negsbj,words[i]))
        self.positiveSubject.extend(self.pList)
        self.negativeSubject.extend(self.nList)
                    
    '''
        [主題詞抓取原則]
        
        找到情緒詞後，先往後找名詞，並確認是否已在清單中
        若找不到或者是找到的詞已在清單中，則再往前找名詞
    '''   
    def getSubject(self, index, sd):
        subject = None
        end = len(sd)-1
        rindex = index
        lindex = index
        # 是否繼續往前找
        keepgoing = True
        # 把目前已有的positiveSubject/negativeSubject合併
        temp = map(operator.itemgetter(0), self.pList) + map(operator.itemgetter(0), self.nList)
        # 往左找
        lcount = 0
        while(lindex > 0):
            lcount += 1
            if lcount > 3:
                break
            lindex -= 1
            word = sd[lindex]
            if word in food:
                if word not in temp:
                    subject = word
                    keepgoing = False
                break
#                 else:
#                     try:
#                         similarWords = model.most_similar(word, topn=10)
#                         bingo = False
#                         for w in similarWords:             
#                             if w in food:
#                                 bingo = True
#                                 break
#                         if word not in temp and bingo:
#                             subject = word
#                             keepgoing = False
#                             break
#                     except KeyError:
#                         pass
        # 若往左找沒找到或者找到的已在清單中，則往後找
        rcount = 0
        if keepgoing:
            while(rindex < end):
                rcount += 1
                if rcount > 3:
                    break
                rindex += 1
                word = sd[rindex]
                if word in food:
                    if word not in temp:
                        subject = word
                        keepgoing = False
                    break
    #             else:
    #                 try:
    #                     similarWords = model.most_similar(word, topn=10)
    #                     bingo = False
    #                     for w in similarWords:             
    #                         if w in food:
    #                             bingo = True
    #                             break
    #                     if word not in temp and bingo:
    #                         subject = word
    #                         keepgoing = False
    #                         break
    #                 except KeyError:
    #                     pass
        return subject

## 測試文章

In [14]:
import json
with open('./Workspace/simpleBlog.json', 'r') as f:
    text = json.load(f)

In [5]:
with open('./Workspace/data/blog_perfect_cut.json', 'r') as f:
    ifoodie = json.load(f)

In [6]:
import time
start = time.time()
output = []
for ele in ifoodie:
    result = {}
    s_analysis = SentimentAnalysis(negdict, posdict, nodict, plusdict, question, ele, ifoodie[ele]['author_id'], food)
    for sentence in ifoodie[ele]['article']:
        s_analysis.sentimentScore(sentence)
    result['_id'] = s_analysis._id
    result['author_id'] = s_analysis.author_id
    result['positiveSubject'] = s_analysis.positiveSubject
    result['negativeSubject'] = s_analysis.negativeSubject
    result['score'] = s_analysis.score
    output.append(result)
print time.time()-start
# with open('./Workspace/data/sentimentResult_20160701.json', 'w') as f:
#     json.dump(output, f)

1044.49092603


In [310]:
with open('./Workspace/ifoodieBlogDic20160619Update5PixnetSmallEnglish.json', 'r') as f:
    blog = json.load(f)

In [313]:
for ele in output:
    blog[ele['_id']]['positiveSubject'] = ele['positiveSubject']
    blog[ele['_id']]['negativeSubject'] = ele['negativeSubject']
    blog[ele['_id']]['score'] = ele['score']

In [314]:
with open('./Workspace/result/BlogWithSubject.json', 'w') as f:
    json.dump(blog, f)

In [41]:
import random
import time
result = {}
ri = random.randint(0,len(ifoodie))
art = ifoodie.values()[ri]['article']
start = time.time()
s_analysis = SentimentAnalysis(negdict, posdict, nodict, plusdict, question, 'b1', 'a1',food)
for sentence in art:
    s_analysis.sentimentScore(sentence)
    result['_id'] = s_analysis._id
result['positiveSubject'] = s_analysis.positiveSubject
result['negativeSubject'] = s_analysis.negativeSubject
result['score'] = s_analysis.score
print time.time()-start
print '情緒總分：%s' % str(s_analysis.score)
for ele in s_analysis.positiveSubject:
    print 'positive', ele[0], ele[1]
for ele in s_analysis.negativeSubject:
    print 'negative', ele[0], ele[1]

0.102883815765
情緒總分：10.5
positive 摃丸 有名
positive 日式拉麵 愛好者
positive 豚骨拉麵 最愛
positive 糖心蛋 愛吃
positive 豚骨拉麵 濃厚
positive 拉麵 品嘗
positive 豚骨拉麵 濃厚
positive 糖心蛋 咬勁
positive 蛋白 滑嫩
positive 拉麵 濃厚
positive 拉麵 濃厚
negative 拉麵 親切
negative 叉燒 滿意


In [42]:
print ifoodie.values()[ri]['_id']
for ele in art:
    print ' '.join(ele)

571782cb51ba803f7fb0101f
新竹市 東區 日式豚骨拉麵 心得 食記 博多 一幸 舍 這天 家人 來到 新竹 玩
說 新竹 最 有名 非 摃丸 米粉 莫屬 每次 吃
想 吃點 樣 手機 搜尋 有家 拉麵 店
身為 日式拉麵 愛好者 最愛 豚骨拉麵
吃 吃 更新 美食 旅遊 情報 來到 新竹市 關新 東路 關新二 街口
博多 一幸 舍 轉角 路上 停車 格還
碰碰運氣 找個 車位 店內 座位 一人 座
幾張 方桌 用餐 不是 用餐 時間
店 內人 不多 省去 等待時間 桌上 水壺
自助 倒水 飲用 菜單
拉麵 一 共有 6 口味
價位 230 小菜 像是 餃子 炸雞 叉燒飯 價錢 要加 一成 服務費
實說 價位 真的 不太 親切 拉麵 客 製 化
包括 選擇 鹹度 麵 軟硬 度 青蔥 拉麵 沒有 糖心蛋
愛吃 糖心蛋 要點 一下 囉 桌上 佐料 添加
拉麵 味道 不夠 加 拉麵 醬 芝麻 黑胡椒 粉
蒜頭 自行 調味 辛子 高菜 跟紅 薑 無限 自助 取用 點好
開放式 廚房 師傅 拉麵 豚 骨 香氣 陣陣 傳來
餓 我點 博多 濃厚 豚骨拉麵
滿滿的 脂泡 配料 一 共有 叉燒 兩片 木耳 青蔥
蛋則 加點 習慣 吃 拉麵 先 品嘗 一口 湯
試試看 濃厚 豚骨拉麵 湯頭 實說 這碗 豚骨拉麵 的湯 算是 濃稠
不到 理想 中 甜度 香氣 叉燒 算是 比較 不 滿意 地方
一幸 舍 叉燒 相當 薄 咬起來 不太 過癮 之外
口感 不夠 軟 香氣 不足
嚼 無味 麵條 則是 極細 麵
很 適合 搭配 豚骨拉麵 吸滿 湯汁 送入 口中
咬勁 不錯 糖心蛋 一整 顆
咬開 爆漿 蛋白 滑嫩 蛋黃 香氣 十足
還挺 滿意 同行 友人 點的紅 拉麵
明太子 做 調味 湯頭 呈現 紅色
喝 微辣 口感
鹹度 這紅 拉麵 湯頭 一 喝 還好
越 喝 越 回味 濃厚 拉麵 更 有特色
比較 記憶 點
一碗 三百元 配料 蔥 木耳 叉燒
單薄 餐 心得 吃 博多 一幸 舍 拉麵 紅 拉麵 濃厚 表現 較佳 說 濃厚 不至於 不好吃 沒有 想 回訪 衝動 價位 不是 很 親切
收取 一成 服務費 算是 一點 小失望 囉 博多 一幸 舍 新竹市 東區 關新 東路 256 號 03 577 3530 11 00 22 00 跟著 尼豪 旅遊 吃 美食


In [298]:
with open('./Workspace/data/testDD.json', 'w') as f:
    json.dump(result, f)

In [299]:
with open('./Workspace/data/testDD.json', 'r') as f:
    dd = json.load(f)
print dd

{u'negativeSubject': [[u'\u9ebb\u8fa3\u71d9', u'\u5f88\u4e45'], [u'\u7d05\u6cb9\u6284\u624b', u'\u5f88\u4e45']], u'_id': u'b1', u'positiveSubject': [[u'\u6c23\u6ce1\u98f2\u6599', u'\u591a\u7a2e'], [u'\u6c99\u62c9', u'\u7368\u4e00\u7121\u4e8c'], [u'\u751f\u9b5a\u7247', u'\u591a\u7a2e'], [u'\u80e1\u9ebb\u8c46\u8150', u'\u81ea\u88fd'], [u'\u80e1\u9ebb\u91ac', u'\u597d\u5403'], [u'\u6c99\u62c9', u'\u5920\u5473'], [u'\u96de\u817f\u8089', u'\u5ae9'], [u'\u9eb5\u98df', u'\u9a5a\u8a1d'], [u'\u6ef7\u8089', u'\u81ea\u88fd'], [u'\u8c46\u8150', u'\u611b\u5403'], [u'\u9d28\u8840', u'\u4e0d\u932f'], [u'\u6284\u624b', u'\u4e0d\u932f'], [u'\u96de\u6392', u'ok'], [u'\u852c\u83dc', u'\u89e3\u81a9'], [u'\u86b5\u4ed4', u'\u98fd\u6eff'], [u'\u863f\u8514', u'\u597d\u5403'], [u'\u751f\u9b5a\u7247', u'\u6700\u611b']], u'score': 63.5}


In [237]:
print ri
for ele in ifoodie.values()[ri]['article']:
    print ' '.join(ele)

61777
再訪 吉祥 小館
主 以時 未 嚐到 菜色 目標 試吃 菜色
細節 詳細 menu 部份 請 初訪 文章 吃 快炒 肯定 飲料
正字 不在
肯定 不會 酒 鍋煲 獅子頭 150 塊 大洋 快炒 吃 獅子頭
湯頭 部份 自然 不能 有太高 期待
白菜 部份 是還 保有 些許 青 脆口 感 獅子頭 總共 四粒
圓滾滾 樣子 還算 飽滿
沒有 美味 湯頭 相輔相成
吃 單純 獅子頭 豬絞肉 肉味
q 軟 口感 中 摻雜 茡 薺 一點點 爽脆
獅子頭 不算 難吃
和湯 白菜
感覺 各走各的路
沒有 太 相得益彰 感覺 蔭鼓 鮮蚵 180 塊 大洋 光看 鮮蚵 這份 量
些許 期待 鮮蚵 鮮度 還滿 不錯
熟度 頗為 恰當
九分熟 調味 不會 有過 厚重
不愛 吃白飯 捧 油
湯汁 拿來 拌飯
一舉兩得 空心菜 炒 90 塊 大洋 實在 很 想點 山菜 類的 吃
可惜 貓 p 太 挑食 無法 接受

空心菜 份量 非常 夠
疊得 高高的 很 爽度
至少 份量 熟度 還滿 恰當
調味 清淡 糖醋魚片 140 塊 大洋 糖醋魚片 味道 不差
調味 是貓 p 最愛 重 芡 重 糖醋
台灣鯛 無土 味 鮮甜 味 自是 不錯
打得 很重 芡 水 魚片 外頭 粉 過於 厚重
吃 膩
算是 小小 可惜 之處 鐵板牛柳 200 塊 大洋 貓 p 最愛 重 口味 鐵板
這道 完全符合 需求 xd 黑胡椒 調味 沒
黑胡椒 辛香 微微 辣 相當 吊人 胃口 丫二訪 吉祥
是因為 點的 菜色 關係
整體 評價 初訪 來得 好些 至少 沒有 吃到 太 失敗 菜色
上次 完食
同行 的貓 p 一改 前言
直說 實在 好吃 太多 重 口味 喜好 上門 沒辦法 說個準
兩次 造訪 沒有 非常 詬病
菜色 部份 價位
不算 差
這間 雨聲 街 這條 算 偏僻 小道 生意 火熱
不是 沒有 道理 3 y
o0 店家 資訊 吉祥 小館 02 28383250 台北市 士林區 雨聲 街 161 號 pm1700
pm2200 a



In [4]:
def deleteBadWords(StrIn):
    Str_BadWords = u'延伸閱讀|連絡方式|電話預約|電話|營業時間|週一|週二|週三|週四|週五|週六|週日|周一|周二|周三|周四|周五|周六|\
                    |周日|假日|公休|平日|地址|粉絲團|星期|禮拜|時間限制|您或許對這些文章有興趣|造訪日期|全年無休|最後點餐|營業|AM|PM|上一篇|下一篇|\
                    |分享此文|您可能喜歡的文章|懶人包|臉書|Facebook|facebook|FB|fb|全世界便宜住宿看這兒|下載愛食記App隨時觀看|按個讚啦|喜歡我的分享嗎|\
                    |瘋台灣民宿網|官方網站|瀏覽人次|最新消息|餐廳名稱|消費時間|無圖文版|網誌|Postedonby|新鮮關注回聲|Christabelle的藝想世界部落格由製作|\
                    |也許對這些文章也有興趣|發表迴響|電子郵件|必要欄位標記|電子郵件|個人網站|輸入圖片顯示文字好證明你不是機器人|站內搜尋分類|最新動態|\
                    |並不會被公開|你的位址 |迴響名稱|用餐日期|留言|載入中|文章文章|粉絲頁|發表|每人平均價位|按個讚|推薦你閱讀|Instagram|instagram|\
                    |美食地圖|版權所有|網友回應|歡迎加入|標籤|著作權聲明|非經授權|不得轉載'
    strClean = re.sub(Str_BadWords,'',StrIn)
    return strClean

def EnglishFullToHalf(StrIn):
    def transform(ele):
        alphabetInt = int(repr(ele.group('number'))[4:8],16)
        transAlphabeInt = alphabetInt - 65248
        return binascii.a2b_hex(hex(transAlphabeInt)[2:4])
    pattern = re.sub(u'(?P<number>[\uff21-\uff3a\uff41-\uff5a])', transform, StrIn)
    return pattern

def setBreakPoint(StrIn):
    pattern = u'[,，.。~～!！?？；]+'
    result = re.sub(pattern, ' ', StrIn)
    return result

def retain_English_Chinese_Arabic_numerals(StrIn):
    Str_English_Chinese = u'([^ 0-9a-zA-Zａ-ｚＡ-Ｚ\u4E00-\u9FCC]+)'
    #Str_English_Chinese = u'([^a-z^A-Z^ａ-ｚ^Ａ-Ｚ^^0-9^０-９^\u3105-\u3129^\u4E00-\u9FCC]+)'
    #\u3105-\u3129為所有注音符號 
    #\u4E00-\u9FCC為所有中文
    strClean = re.sub(Str_English_Chinese,'',StrIn)
    return strClean

def removeSentimentInAds(StrIn):
    pattern = u'.*(喜歡|推薦|喜愛).{0,6}(文章|本文|介紹)'
    def sub(match):
        string = match.group(0)
        type1 = match.group(1) if match.group(1) else ''
        r = string.replace(type1,'')
        return r
    result = re.sub(pattern, sub, StrIn)
    return result

In [5]:
import requests
from bs4 import BeautifulSoup as bs

def removeTag(soup,tag):
    [x.extract() for x in soup.select(tag)]

res = requests.get('https://banbi.tw/2014-01-21-381/')
res.encoding = 'utf-8'
soup = bs(res.text,'html.parser')
removeTag(soup,'script')
removeTag(soup,'a')
removeTag(soup,'.rank')
removeTag(soup,'iframe')
xuite = soup.select('.blogbody')
pixnet = soup.select('.article-content-inner')
ifoodie = soup.select('#blog_content')
miha = soup.select('#content article p')
banbi = soup.select('article')
art = banbi
line = [a.text for a in art if a.text!=""]
st = "".join("".join(line).split()).replace(u'延伸閱讀','').replace('^','')
st = deleteBadWords(st)
st = EnglishFullToHalf(st)
st = setBreakPoint(st)
st = retain_English_Chinese_Arabic_numerals(st)
st = st.split()

In [8]:
art = st
s_analysis = SentimentAnalysis(negdict, posdict, nodict, plusdict, question, 'b1', 'a1',food)
for sent in art:
    s_analysis.sentimentScore(list(jieba.cut(sent)))
print '情緒總分：%s' % str(s_analysis.score)
for ele in s_analysis.positiveSubject:
    print 'positive', ele[0], ele[1]
for ele in s_analysis.negativeSubject:
    print 'negative', ele[0], ele[1]

情緒總分：75
positive 乳腐 美食
positive 白饅頭 柔軟
positive 青江菜 多汁
positive 北京烤鴨 很高
positive 烤鴨 愛吃
positive 蝦子 愛吃
positive 醬汁 適中
positive 韭菜 愛吃
positive 鍋餅 酥脆
positive 焢肉 好吃
positive 白饅頭 柔軟
positive 青江菜 多汁
positive 鴨肉 脆口
positive 湯頭 鮮美
positive 白菜 軟嫩
positive 甜品 開心
positive 花生仁 清甜
positive 紅棗 香
positive 水果 鮮甜
negative 韭菜 喜歡
negative 小黃瓜 可惜
negative 杏仁 不喜
negative 紅棗 煮軟


In [21]:
sentence = u'不是非常不難吃'
s_analysis = SentimentAnalysis(negdict, posdict, nodict, plusdict, question, 'b1', 'a1',food)
s_analysis.sentimentScore(list(jieba.cut(sentence)))
print '情緒總分：%s' % str(s_analysis.score)
for ele in s_analysis.positiveSubject:
    print 'positive', ele[0], ele[1]
for ele in s_analysis.negativeSubject:
    print 'negative', ele[0], ele[1]

情緒總分：0


In [53]:
subject = {}
for ele in text: 
    article = text[ele]['article']
    SA = SentimentAnalysis(negdict, posdict, nodict, plusdict, question, ele, text[ele]['author_id'])
    SA.sentimentScore(article)
    obj2dict = SA.__dict__
    [obj2dict.pop(key,None) for key in ['negdict','posdict','nodict','plusdict', 'question']]
    subject[ele] = obj2dict

## 丟檔案

In [55]:
with open('./Workspace/data/subject.json', 'w') as f:
    json.dump(subject,f)